In [10]:
# Import packages
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
# Print version
print(tf.__version__)

1.12.0


In [12]:
# Get data from URL endpoint
def get_data(s3_endpoint = 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/'):
    
    data = input_data.read_data_sets('data/fashion', 
                                     source_url = s3_endpoint, 
                                     one_hot = True)
    
    return(data)

In [13]:
# Create placeholders for data
def get_data_placeholders(batch_size):
    
    X = tf.placeholder(tf.float32, shape=[batch_size, 784])
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10])
    
    return(X,Y)

In [14]:
# Define forward pass function
def forward(X, arch, batch_size):
    
    image = tf.reshape(X, [batch_size, 28, 28, 1])
    hidden_layer = image
    
    for size in arch:
        num_filters, dim_filters, stride = size[0], size[1], size[2]
        hidden_layer = layers.conv2d(hidden_layer, num_filters, dim_filters, stride, "Same")
        convOut= tf.nn.relu(hidden_layer)
    
    new_shape = tf.divide(tf.size(hidden_layer), batch_size)
    hidden_layer = tf.reshape(hidden_layer, [batch_size, new_shape])
    prbs = layers.fully_connected(hidden_layer, 10, tf.nn.softmax)
    
    return(prbs)

In [15]:
# Define loss function
def get_loss(Y, probs):
    
    loss = tf.reduce_mean(-1*tf.reduce_sum(Y*tf.log(probs), reduction_indices=[1]))
    
    return(loss)

In [16]:
# Get accuracy
def get_accuracy(prbs, ans):
    
    correct = tf.equal(tf.argmax(prbs,1), tf.argmax(ans,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    return(accuracy)

In [17]:
# Get graph
def get_graph(batch_size, num_epochs, arch, learning_rate):
    
    X, Y = get_data_placeholders(batch_size)
    probs = forward(X, arch, batch_size)
    loss = get_loss(Y, probs)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    nodes = [X, Y, probs, loss, optimizer]
    
    return(nodes)

In [18]:
# Define training method
def train_model(batch_size, num_epochs, nn_arch, learning_rate):
    
    # Get nodes
    graph_nodes = get_graph(batch_size, num_epochs, nn_arch, learning_rate)
    
    # Unpack
    X = graph_nodes[0] 
    Y = graph_nodes[1] 
    
    probs = graph_nodes[2] 
    loss = graph_nodes[3] 
    optimizer = graph_nodes[4] 
    
    # Get accuracy graph
    accuracy = get_accuracy(probs, Y)
    
    # Start session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # Train
    total_acc = 0
    
    for i in range(num_epochs):
        print("The epoch is {}".format(i))
        imgs, labels = train.next_batch(batch_size)
        _, acc = sess.run([optimizer,accuracy], feed_dict={X: imgs, Y: labels})
        total_acc +=acc
        
    print('The training accuracy is {}'.format(total_acc/num_epochs))
    
    # Get predictions on dev. set
    total_acc = 0
    
    for i in range(num_epochs):
        imgs, labels = dev.next_batch(batch_size)
        acc = sess.run(accuracy, feed_dict={X: imgs, Y: labels})
        total_acc += acc
    
    print('The dev. accuracy is {}'.format(total_acc/num_epochs))
        
    
    # Get predictions on test set
    total_acc = 0
    
    for i in range(num_epochs):
        imgs, labels = test.next_batch(batch_size)
        total_acc+=sess.run(accuracy, feed_dict={X: imgs, Y: labels})
    
    print('The test accuracy is {}'.format(total_acc/num_epochs))

In [19]:
# Execute
if __name__ == '__main__':
    
    # Set a seed to make sure results are reproducible
    tf.set_random_seed(1234)

    # Get data
    data = get_data()
    
    # Get training, dev., and test sets
    train, dev, test = data.train, data.validation, data.test
    
    # Set parameters
    batch_size = 10
    num_epochs = 100
    nn_arch = [[32, [2,2], 1], [16, [2,2], 1], [12, [2,2], 1], [10, [2,2], 1]]
    learning_rate = 0.2
    
    # Train
    train_model(batch_size, num_epochs, nn_arch, learning_rate)

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
The epoch is 0
The epoch is 1
The epoch is 2
The epoch is 3
The epoch is 4
The epoch is 5
The epoch is 6
The epoch is 7
The epoch is 8
The epoch is 9
The epoch is 10
The epoch is 11
The epoch is 12
The epoch is 13
The epoch is 14
The epoch is 15
The epoch is 16
The epoch is 17
The epoch is 18
The epoch is 19
The epoch is 20
The epoch is 21
The epoch is 22
The epoch is 23
The epoch is 24
The epoch is 25
The epoch is 26
The epoch is 27
The epoch is 28
The epoch is 29
The epoch is 30
The epoch is 31
The epoch is 32
The epoch is 33
The epoch is 34
The epoch is 35
The epoch is 36
The epoch is 37
The epoch is 38
The epoch is 39
The epoch is 40
The epoch is 41
The epoch is 42
The epoch is 43
The epoch is 44
The epoch is 45
The epoch is 46
The epoch is 47
The epoch is 48
The epoch is 49
The epoc